In [172]:
!pip install -q selenium lxml tldextract

In [1]:
import re, os
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
from collections import Counter, defaultdict
from tqdm.auto import tqdm
from urllib.parse import urlparse
from requests import get, head
from random import sample, choice, seed
from time import sleep
from selenium import webdriver
import json
from xmlr import xmlparse, xmliter, XMLParsingMethods
from urllib.parse import urlparse
from multiprocessing import Pool
from functools import partial 

PATTERN_DOMAIN = '(?:http.*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*'

import tldextract
import langdetect
import json
from xml.etree import ElementTree
# import lxml

In [2]:
def extract_suffix(url):
    obj = tldextract.extract(url)
    return obj.suffix


In [3]:
# test
def test_extract_suffix(): 
    actual = extract_suffix("google.com")
    print(actual)
    actual = extract_suffix("www.google.co.th")
    print(actual)
    actual = extract_suffix("www.google.com:433")
    print(actual)
    actual = extract_suffix("https://www.google.com:433")
    print(actual)
test_extract_suffix()

com
co.th
com
com


In [4]:
def extract_domain(url):
    # t = urlparse(url)
    ext = tldextract.extract(url)
    return '.'.join(part for part in ext[:3] if part)


In [5]:
# test
def test_extract_domain(): 
    actual = extract_domain("google.com")
    print(actual)
    actual = extract_domain("www.google.com")
    print(actual)
    actual = extract_domain("www.google.com:433")
    print(actual)
    actual = extract_domain("https://www.google.com:433")
    print(actual)
test_extract_domain()

google.com
www.google.com
www.google.com
www.google.com


In [6]:
# dataset = {
#     "en": defaultdict(lambda: []),
#     "de": defaultdict(lambda: [])
# }

In [7]:
# counter = Counter()
# with tqdm(total=204522994) as pbar_items:
#     with tqdm(total=67977) as pbar_domains:

#         for d in xmliter('../data/en-de.bicleaner07.tmx', 'tu'):
# #             print(d)
            
#             counter['row_counter'] += 1
#             en_url = d['tuv'][0]['prop']
#             de_url = d['tuv'][1]['prop']

#             # print("en_urls", en_url)
#             # print("de_urls", de_url)

#             if en_url == "unknown" or de_url == "unknown":
#                 counter['unkown_url_counter'] += 1
#                 continue

#             if type(en_url) == list:
#                 en_url = en_url[0]
#             if type(de_url) == list:
#                 de_url = de_url[0]

#             if type(en_url) == str:
#                 en_domain = extract_domain(en_url)
#                 # if not en_domain in dataset['en'].keys():
#                     # dataset['en'][en_domain] = []
#                 # dataset['en'][en_domain].append(en_url)
#                 # else:
#                 if len(dataset['en'][en_domain] ) == 10:
#                     continue
#                 dataset['en'][en_domain].append(en_url)

#             if type(de_url) == str:
#                 de_domain = extract_domain(de_url)

#                 # if not de_domain in dataset['de'].keys():
#                 #     dataset['de'][de_domain] = []
#                 # dataset['de'][de_domain].append(de_url)
#                 # else:
#                 if len(dataset['de'][de_domain]) == 10:
#                     continue
#                 dataset['de'][de_domain].append(de_url)


#             if len(dataset['de'].keys()) > counter['domain_counter']:
#                 counter['domain_counter'] += 1           
#                 pbar_domains.update(1)
                
#             pbar_items.update(1)
#             break

In [8]:
# Dump to JSON
with open('../data/en-de.domains.urls.json', 'w', encoding="utf-8") as f:
    
    json.dump(dataset, f, ensure_ascii=False)

NameError: name 'dataset' is not defined

In [9]:
# list(dataset['de'].keys())[-100:]

In [10]:
# len(set(list(dataset['de'].keys())))

## 2. Explore



In [11]:
# Load dataset
with open('../data/en-de.bicleaner07.json', 'r', encoding="utf-8") as f:
    
    %time dataset = json.load(f)

CPU times: user 12.8 s, sys: 21.3 s, total: 34.2 s
Wall time: 38.1 s


In [12]:
def get_status(url):
    try:
        r = head(url, timeout=5)
        code = r.status_code
    except:
        code = 0  # probably bad domain name
    return code

def get_content(url):
    try:
        r = get(url)
#         print(r.text)
#         soup = BeautifulSoup(r.content, 'html.parser')
        text = r.text 
    except Exception as e:
        print(e)
        text = ""  # probably bad domain name
    
    return text


def substitute_de_to_th(de_url):
    pattern = re.compile(r'\bde\b')

    n = len(pattern.findall(de_url))
    if n == 1:

        th_url = pattern.sub("th", de_url)
        return th_url
    else:
        return None

### Find unique domains (registered domain) 

[de]

In [13]:
registered_domains = set()

for domain, urls in dataset['de'].items():
    
    registered_domain = tldextract.extract(domain).registered_domain
    registered_domains.add(registered_domain)    
    

for domain, urls in dataset['en'].items():
    
    registered_domain = tldextract.extract(domain).registered_domain
    registered_domains.add(registered_domain)    

In [14]:
len(registered_domains)

62716

In [15]:
seed(1)
sample(registered_domains, 10)

['rocktimes.de',
 'fixpoints.de',
 'ines-papert.de',
 'hoteljasmin.cz',
 'frogsorbing.com',
 'moser-systemelektrik.de',
 'heyner-germany.de',
 'procab.net',
 'alfred-graf-hh.de',
 'paloma.si']

### Find domains (full domain name)

In [16]:
full_domains = set()

for domain, urls in dataset['de'].items():
    full_domains.add(domain)    
    
for domain, urls in dataset['de'].items():
    full_domains.add(domain)    
    

In [17]:
len(full_domains)

77376

In [18]:
seed(1)
sample(full_domains, 10)

['www.brinkmann-audio.de',
 'www.coloradamar.com',
 'www.wshhhp.com',
 'elagpochetta.com',
 'xmp.net',
 'www.dagstuhl.de',
 'www.planearium.de',
 'www.albergovillafiorita.com',
 'mamolimodel.com',
 'www.bootlock.dk']

### Find unique domains extension statistics

In [19]:
suffix_counter = Counter()

for domain, urls in dataset['de'].items():
    
    suffix = tldextract.extract(domain).suffix
    suffix_counter[suffix] += 1
    
    
# for domain, urls in dataset['en'].items():
    
#     suffix = tldextract.extract(domain).suffix
#     suffix_counter[suffix] += 1
    
    


In [20]:
suffix_counter.most_common(15)

[('com', 26360),
 ('de', 20093),
 ('org', 3584),
 ('net', 3403),
 ('ch', 2941),
 ('it', 2830),
 ('at', 2426),
 ('cz', 1876),
 ('eu', 1864),
 ('biz', 1091),
 ('nl', 981),
 ('info', 908),
 ('pl', 719),
 ('hu', 691),
 ('sk', 499)]

### Find which domains fall in `\bde\b` pattern

In [21]:
SUB_LANG_PATTERNS = [
    (r'(lang=)(de)', r'\1th'),
    (r'(/)(de)(/)', r'\1th\3'),
    (r'(/)(german|ger|ge)(/)', r'\1th\3'),
    (r'(lang=)((german|ger|ge))', r'\1th'),
]

In [22]:
# มีความหลากหลายใน url format จริงๆ สงสัยต้องไปดู code เฉพาะของเขาแล้วมั้ง น่าจะมี handcraft เยอะอยู่
# ดู en ไปแล้ว ดู mt บ้าง

counter = Counter()

res = []

examples_urls_in_pattern = defaultdict(set)

for domain, urls in tqdm(dataset['de'].items()):
    
    for pattern in SUB_LANG_PATTERNS:
        for url in urls[:1]:
            if re.search(pattern[0], url):
                counter[pattern[0]] += 1

                examples_urls_in_pattern[pattern[0]].add(url)

print('number of URLs that matched with the following patterns:')
counter


number of URLs that matched with the following patterns:


Counter({'(/)(de)(/)': 25558,
         '(/)(german|ger|ge)(/)': 1319,
         '(lang=)(de)': 2498,
         '(lang=)((german|ger|ge))': 183})

In [23]:
for pattern, urls in examples_urls_in_pattern.items():
    
    print('\npattern:', pattern)
    print('urls (10 samples)', '\n - '.join(list(urls)[:10]))
    print('')


pattern: (/)(de)(/)
urls (10 samples) http://medjugorjeaccommodation.com/de/hotels/hotel-accommodation/hotel-merci-medugorje-best-hotel.html
 - https://www.hotelbellaromagna.it/de/servizi.php
 - http://www.villaggioinnamorata.com/de/elba.php
 - anikavavic.com/de/presse/story/705/der-energische-ton-der-anika-vavic-von-bach-bis-ravel.html
 - http://www.exclosiv.de/de/toilettenvermietung/modelle/modell_nobles_media.html
 - http://www.creationwatches.com/products/de/tissot-women-s-345/
 - http://www.wheelabratorgroup.com/de/sites/wheelabrator/content/brands/tilghman_wheelabrator.aspx
 - rhizopon.nl/de/anwendungen-und-kulturinformation/verwoehnen-sie-ihre-stecklinge
 - www.globecharge.com/de/contact.html
 - https://zahnarztzentrum.ch/de/baden.html


pattern: (/)(german|ger|ge)(/)
urls (10 samples) http://www.absolute-cold.com/german/agb.htm
 - http://www.gerald-able.de/german/personal.php
 - http://www.austriacar.com/ge/sitemap.htm
 - www.barcodefor.us/barcode-generator/german/card-label-m

In [24]:
## Send http HEAD Request to that modified URLs ,

### 3) Substitute /de/ as /th/, call a HTTP HEAD request and check returned HTTP[link text](https:// [link text](https://)) staus


In [25]:
SUB_LANG_PATTERNS = [
    (r'(lang=)(de)', r'\1th'),
    (r'(/)(de)(/)', r'\1th\3'),
    (r'(/)(german|ger|ge)(/)', r'\1th\3'),
    (r'(lang=)((german|ger|ge))', r'\1th'),
]

In [26]:
def test_substitue_lang():
    urls = [
        "balozi.lv/pub/index.php?lang=ger",
        "balozi.lv/pub/index.php?lang=ge",
        "balozi.lv/pub/index.php?lang=german",
        "balozi.lv/de/index.php?lang=de",
        "balozi.lv/ge/index.php"
    ]
    for url in urls:
        for i in range(0, len(SUB_LANG_PATTERNS)):
            sub_pattern = SUB_LANG_PATTERNS[i]
            match, replace = sub_pattern

            modified_url = re.sub(match, replace, url)

            print('sub_pattern={},modified_url={}, url={}'.format(i,modified_url, url))
            
        print('')
        
test_substitue_lang()

sub_pattern=0,modified_url=balozi.lv/pub/index.php?lang=ger, url=balozi.lv/pub/index.php?lang=ger
sub_pattern=1,modified_url=balozi.lv/pub/index.php?lang=ger, url=balozi.lv/pub/index.php?lang=ger
sub_pattern=2,modified_url=balozi.lv/pub/index.php?lang=ger, url=balozi.lv/pub/index.php?lang=ger
sub_pattern=3,modified_url=balozi.lv/pub/index.php?lang=th, url=balozi.lv/pub/index.php?lang=ger

sub_pattern=0,modified_url=balozi.lv/pub/index.php?lang=ge, url=balozi.lv/pub/index.php?lang=ge
sub_pattern=1,modified_url=balozi.lv/pub/index.php?lang=ge, url=balozi.lv/pub/index.php?lang=ge
sub_pattern=2,modified_url=balozi.lv/pub/index.php?lang=ge, url=balozi.lv/pub/index.php?lang=ge
sub_pattern=3,modified_url=balozi.lv/pub/index.php?lang=th, url=balozi.lv/pub/index.php?lang=ge

sub_pattern=0,modified_url=balozi.lv/pub/index.php?lang=german, url=balozi.lv/pub/index.php?lang=german
sub_pattern=1,modified_url=balozi.lv/pub/index.php?lang=german, url=balozi.lv/pub/index.php?lang=german
sub_pattern=2,m

In [ ]:
urls_with_status = defaultdict(set)
pattern_counter = defaultdict(Counter)

In [ ]:
def substitue_lang_worker(match, replace, url):
#     print('.')
    modified_url = re.sub(match, replace, url)
    status = get_status(modified_url)
#     print('status={}, modified_url={}, url={}'.format(status, modified_url, url))
    
    if int(status) == 200:
        print('status={}, modified_url={}, url={}'.format(status, modified_url, url))
        print('count', pattern_counter[match][200])
#     global_counter+=1
#     print(global_counter, sep=' ', end='', flush=True)

    pattern_counter[match][status] += 1
    urls_with_status[status].add(url)

In [ ]:
_substitue_lang_worker = partial(substitue_lang_worker,
                                     SUB_LANG_PATTERNS[0][0], SUB_LANG_PATTERNS[0][1])
_substitue_lang_worker('g?lang=de')

In [ ]:

for pattern, urls in examples_urls_in_pattern.items():
    for sub_pattern in SUB_LANG_PATTERNS:
        if pattern == sub_pattern[0]:
            match, replace = sub_pattern[0], sub_pattern[1]
            break
            
    print('')
    print('pattern:', pattern)
    print('number of urls for this pattern:', len(urls))
    print('')
    _substitue_lang_worker = partial(substitue_lang_worker,
                                     match, replace)

    for url in tqdm(urls):
        _substitue_lang_worker(url)
#     workers = Pool(processes=1)

#     workers.map(_substitue_lang_worker, urls)
#     workers.close()
#     workers.join()

In [ ]:
pattern_counter

In [228]:
urls_with_status

defaultdict(set, {0: {'g?lang=de'}})

In [133]:
# http_status_de = {}
# counter = 0
# for (domain, url) in tqdm(to_substitute_urls):
# #     urls = urls[:1]
#     urls_th = substitute_de_to_th(url)
# #     urls_th = list(filter(lambda x: x != None, urls_th))
    
#     http_status_de[domain] = { }
#     http_status_de[domain]['status'] = get_status(url)
#     http_status_de[domain]['th_url'] = url
    
# #     if counter == 100:
# #         break

#     if http_status_de[domain]['status'] == 200:
        
#         # Detect lang
# #         print('urls_th[0]', urls_th[0])
# #         content = get_content(urls_th[0])
        
# #         if content == '':
# #             http_status_de[domain]['lang_id'] = None
# #         else:
# #             print(content[:100])
# #             lang = langdetect.detect(content)
# #             print("Language detect:", lang)
# #             http_status_de[domain]['lang_id'] = lang

#         print(http_status_de[domain])

#     counter += 1
#     # break



In [80]:
cnt = Counter()
for item, v in http_status_de.items():
#     if v['status'][0] == 200:
    cnt[v['status'][0]] += 1

In [81]:
cnt

Counter({0: 26999,
         301: 629,
         500: 29,
         200: 179,
         302: 131,
         404: 343,
         403: 29,
         406: 23,
         521: 2,
         307: 2,
         410: 2,
         424: 1,
         409: 1,
         308: 1,
         503: 3,
         402: 1,
         303: 5,
         412: 1})

### Examples:

- https://2fish.co/th/church/does-my-church-really-matter/

- http://www.oil-painting-online.com/th/expressionism-oil-painting/li-shan-oil-painting-art.html

- https://golfauvietnam.com/orient-golf/?lang=th

- https://gsuite.google.se/intl/th/terms/sla.html

- http://www.xd-cinema.com/th/analytical-comparison-3d-4d-5d-6d-7d-9d-truck-mobile-cinema/

## Detect Language

In [83]:
def fetch_content(url):
    """fetch HTML content (excluded HTML <script> tags)"""
    pass
def detect_language(content):
    """return language id"""
    pass

In [84]:
cnt = Counter()
for item, v in http_status_de.items():
    if v['status'][0] == 200:
        url = v['th_url'][0]
        
        content = fetch_content(url)
        
        